In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'/home/img/Code/Python/AICourse/AILabs/lib/python3.13/site-packages/pyspark'

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
        .appName("Titanic Data")\
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 14:19:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
df = (spark.read
.format("csv")
.option("header","true")
.load("data/titanic/train.csv"))

In [6]:
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [7]:
from pyspark.sql.functions import col, isnull, count, when

In [8]:
dataset = df.select(col("Survived").cast("float"),
col("Pclass").cast("float"),
col("Sex"),
col("Age").cast("float"),
col("Fare").cast("float"),
col("Embarked"))

In [9]:
dataset.select([count(when(isnull(c),c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|177|   0|       2|
+--------+------+---+---+----+--------+



In [10]:
dataset = dataset.replace("?", None)\
    .dropna(how="any")

In [11]:
dataset.select([count(when(isnull(c),c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|  0|   0|       0|
+--------+------+---+---+----+--------+



In [12]:
dataset.show(3)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
+--------+------+------+----+-------+--------+
only showing top 3 rows



In [13]:
from pyspark.ml.feature import StringIndexer

In [14]:
dataset = StringIndexer(inputCol="Sex",
outputCol = "Gender",
handleInvalid="keep").fit(dataset).transform(dataset)
dataset = StringIndexer(inputCol="Embarked",
outputCol = "Boarded",
handleInvalid="keep").fit(dataset).transform(dataset)

In [15]:
dataset.show(3)

+--------+------+------+----+-------+--------+------+-------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|Gender|Boarded|
+--------+------+------+----+-------+--------+------+-------+
|     0.0|   3.0|  male|22.0|   7.25|       S|   0.0|    0.0|
|     1.0|   1.0|female|38.0|71.2833|       C|   1.0|    1.0|
|     1.0|   3.0|female|26.0|  7.925|       S|   1.0|    0.0|
+--------+------+------+----+-------+--------+------+-------+
only showing top 3 rows



In [19]:
dataset = dataset.drop("Sex")
dataset = dataset.drop("Embarked")

In [20]:
dataset.show(3)

+--------+------+----+-------+------+-------+
|Survived|Pclass| Age|   Fare|Gender|Boarded|
+--------+------+----+-------+------+-------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|
+--------+------+----+-------+------+-------+
only showing top 3 rows



In [21]:
from pyspark.ml.feature import VectorAssembler

In [22]:
feature_col = ["Pclass", "Age", "Fare", "Gender", "Boarded"]
assembler = VectorAssembler(inputCols=feature_col, outputCol="features")
transformed_data = assembler.transform(dataset)

In [23]:
transformed_data.show(5)

+--------+------+----+-------+------+-------+--------------------+
|Survived|Pclass| Age|   Fare|Gender|Boarded|            features|
+--------+------+----+-------+------+-------+--------------------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|[3.0,22.0,7.25,0....|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|[1.0,38.0,71.2833...|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|[3.0,26.0,7.92500...|
|     1.0|   1.0|35.0|   53.1|   1.0|    0.0|[1.0,35.0,53.0999...|
|     0.0|   3.0|35.0|   8.05|   0.0|    0.0|[3.0,35.0,8.05000...|
+--------+------+----+-------+------+-------+--------------------+
only showing top 5 rows



In [24]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])
print(f"Numberof training samples {training_data.count()}")
print(f"Number of test samples {test_data.count()}")

Numberof training samples 575
Number of test samples 137


In [34]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="Survived", featuresCol="features", maxDepth=5)
model = rf.fit(training_data)

In [35]:
predictions = model.transform(test_data)
predictions.show(5)

+--------+------+----+--------+------+-------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass| Age|    Fare|Gender|Boarded|            features|       rawPrediction|         probability|prediction|
+--------+------+----+--------+------+-------+--------------------+--------------------+--------------------+----------+
|     0.0|   1.0|19.0|   263.0|   0.0|    0.0|[1.0,19.0,263.0,0...|[11.5658182556852...|[0.57829091278426...|       0.0|
|     0.0|   1.0|30.0|   27.75|   0.0|    1.0|[1.0,30.0,27.75,0...|[12.6596893127141...|[0.63298446563570...|       0.0|
|     0.0|   1.0|38.0|     0.0|   0.0|    0.0|(5,[0,1],[1.0,38.0])|[14.0018377566087...|[0.70009188783043...|       0.0|
|     0.0|   1.0|38.0|153.4625|   0.0|    0.0|[1.0,38.0,153.462...|[7.52685087183294...|[0.37634254359164...|       1.0|
|     0.0|   1.0|40.0|     0.0|   0.0|    0.0|(5,[0,1],[1.0,40.0])|[14.1903747098516...|[0.70951873549257...|       0.0|
+--------+------+----+--------+-

In [40]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [41]:
evaluator = MulticlassClassificationEvaluator(labelCol="Survived",predictionCol="prediction", metricName="accuracy")

In [43]:
accuracy = evaluator.evaluate(prediction)
print(f"Model accuracy is : {accuracy}")

Model accuracy is : 0.8248175182481752
